In [7]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt

# Configuration to prevent scientific notation
pd.set_option('display.float_format', lambda x: '%.6f' % x)

def fetch_and_process_data(ticker, start_date="2024-01-01", end_date="2024-12-31", train_ratio=0.8):
    """Fetch and process data with special handling for low-price cryptocurrencies."""
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty or "Adj Close" not in data:
            raise ValueError(f"No valid data found for {ticker}")
        
        close = data["Adj Close"]
        split_index = int(len(close) * train_ratio)
        
        # Handle micro-priced coins  (To be fixed later)
        price_scale = 1000 if close.mean() < 0.1 else 1
        scaled_close = close * price_scale
        
        # Calculate returns based on price level
        if close.mean() > 0.1:
            returns = scaled_close.pct_change().dropna()
        else:
            returns = np.log(scaled_close / scaled_close.shift(1)).dropna()
        
        train_returns = returns.iloc[:split_index]
        test_returns = returns.iloc[split_index:]
        
        scaler = StandardScaler()
        scaler.fit(train_returns.values.reshape(-1, 1))
        
        train_std = scaler.transform(train_returns.values.reshape(-1, 1))
        test_std = scaler.transform(test_returns.values.reshape(-1, 1))
        
        std_returns = np.concatenate([train_std, test_std]).flatten()
        
        return close[returns.index], pd.Series(std_returns, index=returns.index), split_index
    
    except Exception as e:
        raise RuntimeError(f"Error processing {ticker}: {str(e)}")

def generate_signals(returns, split_index, epochs=50):
    """Generate trading signals with proper train-test separation."""
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # GPR Trend Estimation
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    kernel = Matern32()
    gpr = GPR(data=(X_train, y_train), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    
    trend_train = gpr.predict_f(X_train)[0].numpy().flatten()
    trend_test = gpr.predict_f(X_test)[0].numpy().flatten()
    trend = np.concatenate([trend_train, trend_test])
    
    # Feature
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    train_features = features[:split_index]
    train_labels = y[:split_index].flatten()
    
    # LSTM Model
    model = Sequential([
        Input(shape=(1, 2)),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    model.fit(
        train_features.reshape((train_features.shape[0], 1, train_features.shape[1])),
        train_labels,
        epochs=epochs,
        batch_size=32,
        verbose=0
    )
    
    # Predict
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features).flatten()
    return np.clip(predicted_signals, -1, 1)

def backtest_strategy(close, positions):
    """Backtest strategy with improved signal thresholds."""
    entries = positions > 0.2
    exits = positions < -0.2
    
    pf = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        exits=exits,
        size=np.abs(positions),
        freq="1D"
    )
    return pf

def backtest_crypto_list(ticker_list, start_date="2024-01-01", end_date="2024-12-31"):
    """Run backtests for multiple cryptocurrencies with error handling."""
    results = {}
    for ticker in ticker_list:
        print(f"\nBacktesting {ticker}...")
        try:
            close, returns, split_index = fetch_and_process_data(ticker, start_date, end_date)
            positions = generate_signals(returns, split_index)
            portfolio = backtest_strategy(close, positions)
            results[ticker] = portfolio
            
            print(f"Total Return: {portfolio.total_return():.2%}")
            print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.2f}")
            print(f"Max Drawdown: {portfolio.max_drawdown():.2%}")
            
        except Exception as e:
            print(f"Skipped {ticker}: {str(e)}")
    return results

def main():
    """Main execution function with filtered ticker list."""
    filtered_tickers = [
        "BTC-USD", "ETH-USD", "XRP-USD", "SOL-USD", "BNB-USD",
        "DOGE-USD", "ADA-USD", "LINK-USD", "AVAX-USD", 
        "XLM-USD", "TON-USD", "HBAR-USD", "LTC-USD",
        "DOT-USD", "BCH-USD", "UNI-USD", "XMR-USD",
        "AAVE-USD", "ETC-USD", "OKB-USD", "FIL-USD"
    ]

    results = backtest_crypto_list(filtered_tickers)
    
    summary = {}
    for ticker, pf in results.items():
        summary[ticker] = {
            'Total Return': pf.total_return(),
            'Sharpe Ratio': pf.sharpe_ratio(),
            'Max Drawdown': pf.max_drawdown()
        }
    
    summary_df = pd.DataFrame(summary).T
    summary_df['Total Return'] = summary_df['Total Return'].map(lambda x: f"{x:.2%}")
    summary_df['Max Drawdown'] = summary_df['Max Drawdown'].map(lambda x: f"{x:.2%}")
    summary_df['Sharpe Ratio'] = summary_df['Sharpe Ratio'].map(lambda x: f"{x:.2f}")
    
    print("\nFinal Backtest Results:")
    print(summary_df.to_markdown(tablefmt="grid"))

if __name__ == "__main__":
    main()

[*********************100%%**********************]  1 of 1 completed


Backtesting BTC-USD...


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


[*********************100%%**********************]  1 of 1 completed

Total Return: 57.35%
Sharpe Ratio: 1.45
Max Drawdown: -23.47%

Backtesting ETH-USD...


...

Total Return: 8.61%
Sharpe Ratio: 0.85
Max Drawdown: -7.18%

Backtesting OKB-USD...


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


[*********************100%%**********************]  1 of 1 completed

Total Return: 9.18%
Sharpe Ratio: 0.64
Max Drawdown: -12.61%

Backtesting FIL-USD...


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Total Return: 1.39%
Sharpe Ratio: 0.45
Max Drawdown: -3.06%

Final Backtest Results:
+----------+----------------+----------------+----------------+
|          | Total Return   |   Sharpe Ratio | Max Drawdown   |
+==========+================+================+================+
| BTC-USD  | 57.35%         |           1.45 | -23.47%        |
+----------+----------------+----------------+----------------+
| ETH-USD  | 23.28%         |           0.73 | -31.28%        |
+----------+----------------+----------------+----------------+
| XRP-USD  | 1.16%          |           1.37 | -0.54%         |
+----------+----------------+----------------+----------------+
| SOL-USD  | 72.46%         |           1.42 | -21.44%        |
+----------+----------------+----------------+----------------+
| BNB-USD  | 70.49%         |           1.55 | -19.98%        |
+----------+----------------+----------------+----------------+
| DOGE-USD | 0.17%          |           1.3